In [1]:
pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from astroquery.cosmosim import CosmoSim

In [3]:
CS = CosmoSim()

In [4]:
CS.login(username="forero",store_password=True)

Please enter password for encrypted keyring: ········


/homecolab/je.forero/.local/lib/python2.7/site-packages/astroquery-0.3.dev2976-py2.7.egg/astroquery/cosmosim/core.py:70: UserWarning: Authenticating forero on www.cosmosim.org...
  warnings.warn("Authenticating {0} on www.cosmosim.org...".format(self.username))
/homecolab/je.forero/.local/lib/python2.7/site-packages/astroquery-0.3.dev2976-py2.7.egg/astroquery/cosmosim/core.py:75: UserWarning: Authentication successful!
  warnings.warn("Authentication successful!")


AttributeError: 'NoneType' object has no attribute 'string'

In [5]:
CS.check_login_status()

/homecolab/je.forero/.local/lib/python2.7/site-packages/astroquery-0.3.dev2976-py2.7.egg/astroquery/cosmosim/core.py:130: UserWarning: Status: You are logged in as forero.
  warnings.warn("Status: You are logged in as {}.".format(self.username))


In [10]:
sql_query="SELECT DISTINCT * FROM Bolshoi.Redshifts ORDER BY snapnum DESC"
job_id = CS.run_sql_query(query_string=sql_query)

In [14]:
headers, data = CS.download(jobid=job_id,format='csv')

In [39]:
def write_query(ix, iy, iz, n_web=256, n_size=1024):
    ratio = n_size/n_web
    query=" select * from Bolshoi.Tweb%d m where floor(m.ix) = %d"%(n_web, floor(ix/ratio))
    query= query+" and floor(m.iy) = %d"%(floor(iy/ratio))
    query= query+" and floor(m.iz) = %d"%(floor(iz/ratio))
    return query

In [25]:
query = write_query(0,0,0)
job_id = CS.run_sql_query(query_string=query)

/homecolab/je.forero/.local/lib/python2.7/site-packages/astroquery-0.3.dev2976-py2.7.egg/astroquery/cosmosim/core.py:204: UserWarning: Job created: 368666848075061
  warnings.warn("Job created: {}".format(self.current_job))


In [26]:
CS.check_job_status(jobid=job_id)

INFO:astropy:Job 368666848075061: COMPLETED


INFO: Job 368666848075061: COMPLETED [astroquery.cosmosim.core]


'COMPLETED'

In [33]:
headers, data = CS.download(jobid=job_id,format='csv')

In [32]:
print headers, data

['row_id', 'm__webId', 'm__ix', 'm__iy', 'm__iz', 'm__phkey', 'm__eigen1', 'm__eigen2', 'm__eigen3', 'm__ev1_x', 'm__ev1_y', 'm__ev1_z', 'm__ev2_x', 'm__ev2_y', 'm__ev2_z', 'm__ev3_x', 'm__ev3_y', 'm__ev3_z'] [[1, 0, 0, 0, 0, 0, 0.13395200669765472, -0.13198000192642212, -0.431863009929657, 0.7281330227851868, 0.6853790283203125, 0.008839000016450882, -0.1409430056810379, 0.13708999752998352, 0.9804800152778625, 0.6707879900932312, -0.7151659727096558, 0.19641900062561035]]


In [36]:
#this defines the range of the ix, iy, iz integers in the simulation
n_bits = 10
n_size = 2**n_bits
lbox = 250.0

In [45]:
halo_data = loadtxt("../data/samples/samplemc1.txt")
x_halo = halo_data[:,0]
y_halo = halo_data[:,1]
z_halo = halo_data[:,2]

ix = int_((x_halo/lbox)*n_size)
iy = int_((y_halo/lbox)*n_size)
iz = int_((z_halo/lbox)*n_size)

n_points = size(x_halo)
jobs = zeros(n_points, dtype='int')
n_points = 10
for i in range(n_points):
    query = write_query(ix[i], iy[i], iz[i], n_web=256, n_size=1024)
    jobs[i] = CS.run_sql_query(query_string=query)
    print(jobs[i])

368667228402893
368667234810341
368667241518440
368667248556538
368667256094610
368667263815392
368667271887363
368667279707118
368667287769778
368667296089270


In [ ]:
all_data = []
for i in range(n_points):
    headers, data = CS.download(jobid=jobs[i],format='csv')
    print data
    all_data.append(data[0])